In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, roc_auc_score
from pathlib import Path

# Basic configuration

# Replace with name of your CSV file
csv_file_name = 'results.csv'
# Define the threshold for considering an image as synthetic
probability_threshold = 0.2

In [ ]:
csv_dir = Path("..", "csv")
input = csv_dir.joinpath(csv_file_name)

# Function to convert logits to probabilities using sigmoid
def logits_to_probabilities(logits):
    return 1 / (1 + np.exp(-logits))

# Load the data from the CSV file
df = pd.read_csv(input)

# Applying sigmoid function to convert logits to probabilities
model_columns = {
    'logits_Grag2021_progan': 'dMDetectorResultsinferenceResults_logits_Grag2021_progan',
    'logits_Grag2021_latent': 'dMDetectorResultsinferenceResults_logits_Grag2021_latent',
    'resnet50nodown_progan': 'gANDetectorResultsinferenceResults_logits_gandetection_resnet50nodown_progan',
    'resnet50nodown_stylegan2': 'gANDetectorResultsinferenceResults_logits_gandetection_resnet50nodown_stylegan2',
    'exif_model': 'exifDetectorResultsinferenceResults_isSyntheticImage'
}

for model_name, column in model_columns.items():
    if 'exif' not in model_name:  # Exif model doesn't need conversion
        df[f'{model_name}_prob'] = logits_to_probabilities(df[column])

def calculate_model_performance(df, threshold):
    performances = []
    for model_name, column in model_columns.items():
        pred_column = f'{model_name}_pred'
        prob_column = f'{model_name}_prob' if 'exif' not in model_name else column
        df[pred_column] = df[prob_column] > threshold
        accuracy = accuracy_score(df['true_label'], df[pred_column])
        auc_roc = roc_auc_score(df['true_label'], df[prob_column]) if 'exif' not in model_name else np.nan
        performances.append({'Model': model_name, 'Accuracy': accuracy, 'ROC AUC': auc_roc})

    # Aggregate results
    prediction_columns = [f'{model}_pred' for model in model_columns.keys()]
    df['final_pred'] = df[prediction_columns].any(axis=1)
    aggregate_accuracy = accuracy_score(df['true_label'], df['final_pred'])
    aggregate_auc_roc = roc_auc_score(df['true_label'], df[prediction_columns].max(axis=1))
    performances.append({'Model': 'aggregate', 'Accuracy': aggregate_accuracy, 'ROC AUC': aggregate_auc_roc})

    return pd.DataFrame(performances)

# Calculate model performance
performances_df = calculate_model_performance(df, probability_threshold)
print("Data loaded successfully!")

In [ ]:
# Print performance as a table
display(performances_df)

In [ ]:
# Setting the plot size
plt.figure(figsize=(12, 6))

# Plotting Accuracy
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
sns.barplot(x='Accuracy', y='Model', hue='Model', data=performances_df, palette="viridis", legend=False)
plt.title('Model Accuracies')
plt.xlabel('Accuracy')
plt.ylabel('Model')
plt.xlim(0, 1)  # Accuracies range from 0 to 1

# Plotting ROC AUC
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
sns.barplot(x='ROC AUC', y='Model', hue='Model', data=performances_df, palette="magma", legend=False)
plt.title('Model ROC AUC')
plt.xlabel('ROC AUC')
plt.ylabel('Model')
plt.xlim(0, 1)  # ROC AUC ranges from 0 to 1

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Plotting a histogram of the probabilities for each model to visualize the distribution
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Probability Distributions per Model')

models = ['dMDetector_progan_prob', 'dMDetector_latent_prob', 'gANDetector_progan_prob', 'gANDetector_stylegan2_prob']
for i, model in enumerate(models):
    ax = axes[i//2, i%2]
    # Ensure the model column exists and contains valid numerical data
    if model in df.columns and pd.api.types.is_numeric_dtype(df[model]):
        sns.histplot(df[model], bins=30, ax=ax, kde=True)
        ax.set_title(model)
        ax.set_xlim(0, 1)  # Probabilities range from 0 to 1
    else:
        ax.set_title(f"{model} - data not available")

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()